### Description:

AES-128 algorithm with a hardcoded key is available as service.

To perform encryption, the binary requires 16-bytes plaintext (in hex):

```
    ../wbcenc --ptext="010203040506070809000a0b0c0d0e1f"
```    
The result is printed to console:
```
    3F7BA33CBAF53C4BBE5E849C3456E6DA 7A9158C14608067002F...B7ACD9201C4A5A6BB3715
```    
where:
```
    3F7BA33CBAF53C4BBE5E849C3456E6DA is a ciphertext
    
    7A9158C14608067002F...B7ACD9201C4A5A6BB3715 is a trace (information left by a programmer)
```
### Tips:

A 'trace' is a side-channel information left by a programmer to debug the implementation.

In this example a trace consists of random information plus at certain position the trace has Hamming weights of a 10th round input state bytes of the AES-128 algorithm, i.e., a State before the Sbox operation in the last round.

### Task:

Your task is to find the Master key (round key 0) embedded into the binary. 

The master key is in the form of HEIG{XXXXXXXXXX}, where X is an ASCII printable symbol.

### Leakage illustration

<img src="support/Slide.png">

In [1]:
import numpy as np
import numpy.matlib as npm
import binascii
import string
import re
import socket

HOST = 'iict-mv330-sfa'
PORT = 4002

import sca_training

#----------------------------------------------------------------------------
# This function calls tested binary ../wbcenc either with a user-defined 
# plaintext (if plaintext satisfies all the requirements) or with a 
# predefined plaintext (the same as in the header above)
#
# INPUTS:
#     plaintext - a string of 32 symbols representing 16 hex bytes of ciphertext
#     verbose   - a flag to print values in the function call or not
# OUTPUTS:
#     output  - a raw binary output
#     ctext   - resulted ciphertext converted to numpy array of uint8
#     trace   - a trace associated with encryption process
#----------------------------------------------------------------------------
def binary_aes128_encrypt(plaintext, verbose=False):
    import subprocess
    pattern = '[0-9A-F]{32,324}'
    
    #Check the correctness of the plaintext
    if all(c in string.hexdigits for c in plaintext) and len(plaintext) == 32:
        
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            s.connect((HOST, PORT))
            s.recv(1024)
            s.sendall(plaintext.encode())
            s.sendall(b'\n')

            output = s.recv(1024)
    
    #Get the ciphertext (first 32 symbols) and the trace (last 32 symbols)
    result = re.findall(pattern, str(output))
    #print(result[0], result[1])
    
    #Transform the result into numpy array
    ctext = np.frombuffer(binascii.unhexlify(result[0]), dtype=np.uint8)
    trace = np.frombuffer(binascii.unhexlify(result[1]), dtype=np.uint8)

    if (verbose):
        print('Binary output:', output.strip())
        print('Ciphertext as numpy array:', ctext)
        print('Trace as numpy array:', trace)
    
    return output, ctext, trace


#----------------------------------------------------------------------------
# This function calls supportive library to compute a master key from
# the last round key.
#
# INPUTS:
#     last_round_key - a numpy array of 16 elements representing 16 hex bytes of
#                      the last round key, i.e., Round Key 10
# OUTPUTS:
#     key_schedule  - 11 round keys of 16 bytes each (key_schedule[0,0,:] is the
#                     master key
#----------------------------------------------------------------------------
# key_schedule = sca_training.inverse_key_expansion(last_round_key)

In [2]:
output, ctext, trace = binary_aes128_encrypt("010203040506070809000a0b0c0d0e1f", verbose=True)
len(trace)

Binary output: b'3F7BA33CBAF53C4BBE5E849C3456E6DA 615D9BE7864E74769115EA1FC83EEFDFF1C2A2E3D3A4AC19A09000833997279AF401C083503D38E1882B3F5AA06D789A6E5A874B3D7264DD419A697A70C61D6D071D2F8D909D03030303060302050304030403030305E70498907E1195D2B49C2E2268BE'
Ciphertext as numpy array: [ 63 123 163  60 186 245  60  75 190  94 132 156  52  86 230 218]
Trace as numpy array: [ 97  93 155 231 134  78 116 118 145  21 234  31 200  62 239 223 241 194
 162 227 211 164 172  25 160 144   0 131  57 151  39 154 244   1 192 131
  80  61  56 225 136  43  63  90 160 109 120 154 110  90 135  75  61 114
 100 221  65 154 105 122 112 198  29 109   7  29  47 141 144 157   3   3
   3   3   6   3   2   5   3   4   3   4   3   3   3   5 231   4 152 144
 126  17 149 210 180 156  46  34 104 190]


100

### ATTACK CODE
Here you need to implement your attack which gives you the last round key.

Once the last round key is found - you need to compute the master key: this can be done with the function get_master_key()

### Programming tips
#### Numpy code
* np.arange(256).astype(np.uint8)
* np.bitwise_xor(m1, m2)
* np.where()

#### Code prepared for training
* sca_training.invSbox[sbox_out]
* sca_training.shift_rows(trace)

#### Various cod
* ''.join('{:02x}'.format(c) for c in int_array)

In [21]:
from sca_training import invSbox, inv_shift_rows
import random

SAMPLES = 64

ctexts = np.zeros((SAMPLES, 16), dtype=np.uint8)
traces = np.zeros((SAMPLES, 100), dtype=np.uint8)

for i in range(SAMPLES):
    o, ct, trace = binary_aes128_encrypt(np.random.bytes(16).hex(), verbose=False)
    ctexts[i, :] = ct
    traces[i, :] = trace

print(ctexts.shape, traces.shape)

t_ctexts = ctexts.T
t_traces = traces.T

print(t_ctexts.shape, t_traces.shape)

k10 = np.zeros((16), dtype=np.uint8)

#shifted_traces = sca_training.shift_rows(traces).T
key_cand = npm.repmat(np.arange(256).astype(np.uint8), SAMPLES, 1)
key = np.zeros(16).astype(np.uint8)

for k_i in range(16):
    sbox_out = np.bitwise_xor(key_cand, ctexts[:, k_i].reshape(SAMPLES, 1))
    inv_sbox = sca_training.invSbox[sbox_out]
    inv_sbox_hw = sca_training.HW_uint8[inv_sbox]

    # Revoir
    for s_i in range(traces.shape[1]):
      best_keys = np.where(np.sum(inv_sbox_hw==traces[:,s_i].reshape(SAMPLES, 1), axis=0) == SAMPLES)[0]

      if len(best_keys) == 1:
          print('Only one key candidate remained for ciphertext byte', k_i)
          key[k_i] = best_keys[0]

      if len(best_keys) > 1:
          print('Several key candidates remained for ciphertext byte', k_i)
          print(best_keys)

key_schedule = sca_training.inverse_key_expansion(key)
print(key_schedule.shape)

print('Master key in hex:', key_schedule[0,0,:])

print('Master key in ASCII:', binascii.unhexlify(''.join('{:02x}'.format(c) for c in key_schedule[0,0,:])))


(64, 16) (64, 100)
(16, 64) (100, 64)
Only one key candidate remained for ciphertext byte 0
Only one key candidate remained for ciphertext byte 1
Only one key candidate remained for ciphertext byte 2
Only one key candidate remained for ciphertext byte 3
Only one key candidate remained for ciphertext byte 4
Only one key candidate remained for ciphertext byte 5
Only one key candidate remained for ciphertext byte 6
Only one key candidate remained for ciphertext byte 7
Only one key candidate remained for ciphertext byte 8
Only one key candidate remained for ciphertext byte 9
Only one key candidate remained for ciphertext byte 10
Only one key candidate remained for ciphertext byte 11
Only one key candidate remained for ciphertext byte 12
Only one key candidate remained for ciphertext byte 13
Only one key candidate remained for ciphertext byte 14
Only one key candidate remained for ciphertext byte 15
(1, 11, 16)
Master key in hex: [ 72  69  73  71 123  67  97 116  99 104 109 101 105 121  99 

In [ ]:
SAMPLES = 5
LEN_TRACES = 100

ctexts = np.zeros((SAMPLES, 16), dtype=np.uint8)
traces = np.zeros((SAMPLES, LEN_TRACES), dtype=np.uint8)

for i in range(SAMPLES):
    o, ct, trace = binary_aes128_encrypt(np.random.bytes(16).hex(), verbose=False)
    ctexts[i, :] = ct
    traces[i, :] = trace

ctexts_cols = ctexts.T
traces_cols = traces.T

key = np.zeros(16).astype(np.uint8)
key_cand = npm.repmat(np.arange(256).astype(np.uint8), SAMPLES, 1).T

for k_i in range(16):
    sbox_out = np.bitwise_xor(key_cand, ctexts_cols[k_i])
    inv_sbox = sca_training.invSbox[sbox_out]
    hw_inv_sbox = sca_training.HW_uint8[inv_sbox]
    
    coeffs = [ np.corrcoef(hw_inv_sbox[k_i], traces_cols[s_i]) for s_i in range(traces.shape[1]) ]
    print(np.argmax(coeffs))

7
0
7
0
7
0
0
0
0
7
0
0
0
0
0
0
